**HOMEWORK 1**

NHÓM 11

Trần Thị Cẩm Yến - 050608200810

Ngô Võ Thị Mai Quyên - 050608200589

Nguyễn Thị Anh Thư - 050608200698

**CÂU 1**

In [ ]:
%%capture
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
     

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Avoids scroll-in-the-scroll in the entire Notebook
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 400})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

Mounted at /content/drive


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

<IPython.core.display.Javascript object>

--2023-04-02 14:48:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  55.1MB/s    in 0.2s    

2023-04-02 14:48:33 (55.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [ ]:
def line_to_friend_ownership(line):
    split = line.split()
    user_id = int(split[0])
    if len(split) == 1:
        friends = []
    else:
        friends = list(map(lambda x: int(x), split[1].split(',')))
    return user_id, friends


<IPython.core.display.Javascript object>

In [ ]:
def friend_ownership_to_connection(f_o):
    user_id = f_o[0]
    friends = f_o[1]
    connections = []
    for friend_id in friends:
        key = (user_id, friend_id)
        if user_id > friend_id:
            key = (friend_id, user_id)
        connections.append((key, 0))  # they are friends, value=0
    for friend_pair in itertools.combinations(friends, 2):
        friend_0 = friend_pair[0]
        friend_1 = friend_pair[1]
        key = (friend_0, friend_1)
        if friend_0 > friend_1:
            key = (friend_1, friend_0)
        connections.append((key, 1))  # they have mutual friends, value=1
    return connections

In [ ]:
def mutual_friend_count_to_recommendation(f):
    pair = f[0]
    friend0 = pair[0]
    friend1 = pair[1]
    noMutFriends = f[1]
    rec0 = (friend0, (friend1, noMutFriends))
    rec1 = (friend1, (friend0, noMutFriends))
    return [rec0, rec1]

In [ ]:
def recommendation_to_sorted_truncated(recs):
    recs.sort(key=lambda x: (-x[1], x[0]))
    return list(map(lambda x: x[0], recs))[:10]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
%matplotlib inline
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

<IPython.core.display.Javascript object>

In [ ]:
# Read from text file 
lines = sc.textFile("/content/soc-LiveJournal1Adj.txt")

# Map each line to the form: (user_id, [friend_id_0, friend_id_1, ...])
friend_ownership = lines.map(line_to_friend_ownership).filter(lambda friend: '' != friend[1])#.filter(lambda friend: 1000> friend[0]) #take 1000 samples for testing

# Map friend ownerships to instances of ((user_id, friend_id), VALUE).
# VALUE = 0 => pairs are already friends.
# VALUE = 1 => pairs have mutual friends.
friend_edges = friend_ownership.flatMap(friend_ownership_to_connection)
friend_edges.cache()

# Filter pairs that are already friends
mutual_friend = friend_edges.groupByKey() \
    .filter(lambda edge: 0 not in edge[1]) \
    .flatMap(lambda x: [(x[0],item) for item in x[1]]) # flat it to count total mutual friends No; use map directly causes bugs

# Count mutual friends by adding up values
mutual_friend_counts = mutual_friend.reduceByKey( lambda x,y : x+y)

# Create the recommendation objects, group them by key, then sort and 
recommendations = mutual_friend_counts.flatMap(mutual_friend_count_to_recommendation).groupByKey() 

# Truncate the recommendations to the 10 most highly recommended.
recommendations10 = recommendations.map(lambda m: (m[0], recommendation_to_sorted_truncated(list(m[1])))).sortByKey() 

# Include in your writeup the recommendations for the users with following user IDs: 924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993.
results = recommendations10.filter(lambda recommendations: recommendations[0] in [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993])


<IPython.core.display.Javascript object>

In [ ]:
    results.collect()

<IPython.core.display.Javascript object>

[(924, [439, 2409, 6995, 11860, 15416, 43748, 45881]),
 (8941, [8943, 8944, 8940]),
 (8942, [8939, 8940, 8943, 8944]),
 (9019, [9022, 317, 9023]),
 (9020, [9021, 9016, 9017, 9022, 317, 9023]),
 (9021, [9020, 9016, 9017, 9022, 317, 9023]),
 (9022, [9019, 9020, 9021, 317, 9016, 9017, 9023]),
 (9990, [13134, 13478, 13877, 34299, 34485, 34642, 37941]),
 (9992, [9987, 9989, 35667, 9991]),
 (9993, [9991, 13134, 13478, 13877, 34299, 34485, 34642, 37941])]

**CÂU 2**

**2a**

Nhược điểm khi bỏ qua Pr(B) là không phải tất cả các luật có độ tin cậy cao đều thú vị. Ví dụ, luật X→sữa có thể có độ tin cậy cao đối với nhiều tập hợp mặt hàng X, vì sữa được mua rất nhiều và độ tin cậy sẽ cao. Trong trường hợp này, sử dụng Interest (A → B) = |conf(A → B) - Pr(B)| là một giải pháp tốt.

Lift không bị ảnh hưởng bởi hạn chế này vì lift(A → B) = conf(A→ B)/S(B). Ví dụ, nếu sự xuất hiện của A không ảnh hưởng đến B, lift(A → B) = 1. Nếu sự xuất hiện của A tăng tần suất của B, lift(A → B) > 1. Nếu sự xuất hiện của A giảm tần suất của B, lift(A → B) <1.

Conviction cũng không bị ảnh hưởng bởi hạn chế này vì conv(A → B) = (1 - S(B) /(1- conf(A → B)). Ví dụ, nếu sự xuất hiện của A không ảnh hưởng đến B, conv(A → B) = 1. Nếu sự xuất hiện của A tăng tần suất của B, conv(A → B) > 1. Nếu sự xuất hiện của A giảm tần suất của B, conv(A → B) <1.


**2b**

Confidence không đối xứng. conf(A → B) = Pr(B|A)= Pr(A∩B)/Pr(A), trong khi đó conf(B → A = Pr(A|B) = Pr(A∩B)/Pr(B). Vì Pr(A) không nhất thiết bằng Pr(B), nên độ đo này không đối xứng.

Lift là đối xứng. lift (A→ B)= conf(A → B)/S(B)= support(A∪B)/(support (A)S(B)) = (Nsupport(A∪B)/ (support (A) support (B)), trong khi đó lift(B→ A) = conf(B → A)/S(A) = support (B∪A)/ (support(B)S(A) = (Nsupport(A∪B)/(support (A)support (B)). Vì lift (A → B) = lift(B → A), nên độ đo này đối xứng.

Conviction không đối xứng. conv(A → B) = (1 - S(B))/(1- conf(A → B)) = (1-Pr(B)/(1 - Pr(AcB)/Pr(A), trong khi conv(B→ A) = (1 -S(A))/(1- conf(B → A) = (1- Pr(A))/(1–Pr(A∩B)/Pr(B)). Vì Pr(A) không nhất thiết bằng Pr(B), nên độ đo này không đối xứng.

**2c**

Độ tin cậy là một chỉ số mong muốn. Giả sử A → B luôn đúng, có nghĩa là B xuất hiện trong mọi giao dịch mà A xuất hiện. Sau đó, theo trực giác, conf(A → B) = Pr(B|A) = Pr(A, B) / Pr(A) = 1. Vì conf(A → B) có khoảng giá trị từ [0, 1], nên chỉ số này là mong muốn.

Lift không mong muốn. Giả sử A → B luôn đúng. Sau đó, lift(A → B) = conf(A → B) / S(B) = 1 / S(B) = N / support (B). Giả sử tồn tại một luật khác luôn đúng dưới dạng C → D và giả định support (D) # support (B). Sau đó, lift(C → D) = conf(C→ D) / S(D) = 1 / S(D) = N / support(D). Vì vậy, lift(A → B) # lift (C → D), điều này cho thấy chỉ số này không mong muốn.

Conviction là một chỉ số mong muốn. Giả sử A → B luôn đúng. Sau đó, conv(A → B) = (1 - S(B)) / (1 - conf(A → B)). Vì conf(A → B) = 1, conv(A → B) = ∞. Vì vô cùng chắc chắn là tối đa, nên chỉ số này là mong muốn.


**2d**

In [ ]:
# Import libraries
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()
sc = spark.sparkContext

<IPython.core.display.Javascript object>

In [ ]:
lines = sc.textFile('/content/browsing.txt')

<IPython.core.display.Javascript object>

In [ ]:
def normalize_group(*args):
    return sorted(args)

<IPython.core.display.Javascript object>

In [ ]:
def get_pairs(items, frequent_items):
    pairs = []
    filtered_items = [item for item in items if item in frequent_items]
    for i, item1 in enumerate(filtered_items[:-1]):
        for item2 in filtered_items[i+1:]:
            pairs.append(normalize_group(item1, item2))
    return pairs

<IPython.core.display.Javascript object>

In [ ]:
def common_elements(list1, list2):
    return list(set(list1) & set(list2))

<IPython.core.display.Javascript object>

In [ ]:
def get_triple(lists):
    unique_elements = set()
    for lst in lists:
        unique_elements.update(lst)
    sorted_elements = tuple(sorted(unique_elements))
    return sorted_elements if sorted_elements else ()

<IPython.core.display.Javascript object>

In [ ]:
frequent_items = (lines.flatMap(lambda line: line.split())
                   .map(lambda ele: (ele, 1))
                   .reduceByKey(lambda count1, count2: count1 + count2)
                   .filter(lambda x: x[1] >= 100))

<IPython.core.display.Javascript object>

In [ ]:
frequent_itemset = frequent_items.collectAsMap()

<IPython.core.display.Javascript object>

In [ ]:
frequent_pairs = (lines.map(lambda line: line.split())
                    .flatMap(lambda lst: get_pairs(lst, frequent_itemset))
                    .map(lambda pair: (tuple(pair), 1))
                    .reduceByKey(lambda count1, count2: count1 + count2)
                    .filter(lambda x: x[1] >= 100))

In [ ]:
freq_pairs = frequent_pairs.map(lambda pair_count: list(pair_count[0])).collect()
freq_pairs_with_count = frequent_pairs.collectAsMap()

<IPython.core.display.Javascript object>

In [ ]:
freq_pair_conf = (frequent_pairs.flatMap(lambda elem: (((elem[0][0], elem[0][1]), elem[1]), ((elem[0][1], elem[0][0]), elem[1])))
                                .map(lambda elem: (elem[0], float(elem[1]*1000/frequent_itemset[elem[0][0]])))
                                .map(lambda elem: (elem[0], elem[1]/1000))
                                .sortBy(lambda x: -x[1]) 
                  )

<IPython.core.display.Javascript object>

In [ ]:
freq_pair_conf.take(5)

<IPython.core.display.Javascript object>

[(('DAI93865', 'FRO40251'), 1.0),
 (('GRO85051', 'FRO40251'), 0.999176276771005),
 (('GRO38636', 'FRO40251'), 0.9906542056074766),
 (('ELE12951', 'FRO40251'), 0.9905660377358491),
 (('DAI88079', 'FRO40251'), 0.9867256637168142)]

**2e**

In [ ]:
lines = sc.textFile('/content/browsing.txt')

<IPython.core.display.Javascript object>

In [ ]:
def normalize_group(*args):
    return (sorted(args))

<IPython.core.display.Javascript object>

In [ ]:
def get_pairs(items, frequent_items):
    pairs = []
    filtered_items = [item for item in items if item in frequent_items]
    for i, item1 in enumerate(filtered_items[:-1]):
        for item2 in filtered_items[i+1:]:
            pairs.append(normalize_group(item1, item2))
    return pairs

<IPython.core.display.Javascript object>

In [ ]:
def common_elements(list1, list2):
    return list(set(list1) & set(list2))

<IPython.core.display.Javascript object>

In [ ]:
def get_triple(lists):
    unique_elements = set()
    for lst in lists:
        unique_elements.update(lst)
    sorted_elements = tuple(sorted(unique_elements))
    return sorted_elements if sorted_elements else ()

<IPython.core.display.Javascript object>

In [ ]:
requent_items = (lines.flatMap(lambda line: line.split())
                   .map(lambda ele: (ele, 1))
                   .reduceByKey(lambda count1, count2: count1 + count2)
                   .filter(lambda x: x[1] >= 100))

<IPython.core.display.Javascript object>

In [ ]:
frequent_itemset = frequent_items.collectAsMap()

<IPython.core.display.Javascript object>

In [ ]:
frequent_pairs = (lines.map(lambda line: line.split())
                    .flatMap(lambda lst: get_pairs(lst, frequent_itemset))
                    .map(lambda pair: (tuple(pair), 1))
                    .reduceByKey(lambda count1, count2: count1 + count2)
                    .filter(lambda x: x[1] >= 100))

<IPython.core.display.Javascript object>

In [ ]:
freq_pairs = frequent_pairs.map(lambda pair_count: list(pair_count[0])).collect()
freq_pairs_with_count = frequent_pairs.collectAsMap()

<IPython.core.display.Javascript object>

In [ ]:
freq_pair_conf = (frequent_pairs.flatMap(lambda elem: (((elem[0][0], elem[0][1]), elem[1]), ((elem[0][1], elem[0][0]), elem[1])))
                            .map(lambda elem: (elem[0], float(elem[1]*1000/frequent_itemset[elem[0][0]])))
                                .map(lambda elem: (elem[0], elem[1]/1000))
                                .sortBy(lambda x: -x[1])     
                  )

<IPython.core.display.Javascript object>

In [ ]:
candidate_triples = (lines.map(lambda line: line.split())
                      .flatMap(lambda lst: combinations(lst, r=3))
                      .map(lambda triple: get_pairs(triple, frequent_itemset))
                      .filter(lambda pairs: len(pairs) == 3)
                      .filter(lambda pairs: all(ele in freq_pairs for ele in pairs))
                      .map(lambda pairs: get_triple(pairs))
                      .map(lambda triple: (triple, 1))
                      .reduceByKey(lambda count1, count2: count1 + count2)
                      .filter(lambda x: x[1] >= 100))

<IPython.core.display.Javascript object>

In [ ]:
freq_triples_conf = (candidate_triples.flatMap(lambda elem: (((elem[0][0],elem[0][1]),elem[0][2],elem[1]), ((elem[0][0],elem[0][2]),elem[0][1],elem[1]), ((elem[0][1],elem[0][2]),elem[0][0],elem[1])))
                                      .map(lambda elem: (elem[0], elem[1], float(elem[2])/float(freq_pairs_with_count[elem[0]]))) \
                                      .sortBy(lambda x: (-x[2],x[0],x[1])))

In [ ]:
freq_triples_conf.take(5)

**3a**

Giả sử x, y, z ∈ S, chúng ta có:

Pr(h(x) ≠ h(z)) = Pr{(h(x) ≠ h(z)) ∩ (h(x) = h(y))} + Pr{(h(x) ≠ h(y)) ∩ (h(x) ≠ h(z))} = Pr(h(y) ≠ h(z)) + Pr{(h(x) ≠ h(y)) ∩ (h(x) = h(z))} ≤ Pr(h(y) ≠ h(z)) + Pr(h(x) ≠ h(y))

Điều này tương đương với:

1 - sim(x,z) ≤ 1 - sim(y,z) + 1 - sim(x,y)

điều này dẫn đến:

d(x,z) ≤ d(y,z) + d(x,y)

**3b**

Cho A = {1,2,3}, B = {4,5,6}, C = {3,4,5,6}. Khi đó, simover(A,B) = 0, simover(B,C) = 3/3 = 1, simover(A,C) = 1/3. Vậy D(A,B) = 1 - 0 = 1, D(B,C) = 1 - 1 = 0, D(A,C) = 1 - 1/3 = 2/3. Ta có D(A,B) = 1 > 2/3 = D(B,C) + D(A,C).

**3c**

Cho A = {1,2,3}, B = {4,5,6}, C = {3,4,5,6}. Khi đó, simdice(A,B) = 0, simdice(B,C) = 3/3.5 = 0.857, simdice(A,C) = 1/3.5 = 0.286. Vậy D(A,B) = 1-0 = 1, D(B,C) = 1-0.857 = 0.14, D(A,C) = 1-0.286 = 0.71. Ta có D(A,B) = 1 > 0.85 = D(B,C) + D(A,C).

**CÂU 4**

**4a**

Đối với mỗi 1 ≤ J ≤ L , và mỗi điểm dữ liệu x thuộc T, Pr[x ∈ T ∩ Wj] ≤ pk2 = 1/n), do đó theo công thức tính tuyến tính của kỳ vọng ta có  E[∑_(j=1)^L▒〖|T ∩Wj|≤L〗
Do đó, ứng dụng của bất đẳng thức Markov đưa ra giới hạn xác suất mong muốn.

**4b**

Vì d(x^*, z) ≤ λ, Pr[∀1 ≤ j ≤ L, g_j(x^*) = g_j(z)] ≥ 〖p_1〗^k
Pr[∀1 ≤ j ≤ L, g_j(x^*) # g_j(z)] = {1 − P r[∀1 ≤ j ≤ L, g_j(x^*) = g_j(z)]} ≤ 1 −〖p_1〗^k =〖(1 - p^(〖log〗_(p_2 ) 1/n))〗^(L ) 〖=(1-n^(-p))〗^L
Do đó 1-x≤e^x 〖,(1-n^(-p) )〗^L≤e^(-L/n^p )=e^(-1)=1/e


**4c**

Có thể xảy ra lỗi nếu (1) có nhiều hơn 3L điểm trong L bộ chứa trong g_j(z) và tất cả 3L điểm được lấy mẫu đều từ tập T
HOẶC (2) một điểm m ∈ A không được băm vào bất kỳ nhóm L nào trong g_j(z)
Pr(lỗi) = Pr(trường hợp 1) + Pr(trường hợp 2) = Pr⁡(trường hợp 1)+Pr⁡(trường hợp2)=Pr[∑_(j=1)^L▒〖|T∩W_j |≥ 3L ] + Pr[∀1 ≤ j ≤ L,g_j (x^* )# g_j (z)≤1/3+1/e 〗
Vì vậy, với xác suất lớn hơn một số hằng số cố định, điểm được báo cáo là một (c, λ)- ANN thực tế

**4d**

In [ ]:
import numpy as np
import random
import time
import pdb
import unittest
from PIL import Image
import matplotlib.pyplot as plt
from typing import List

<IPython.core.display.Javascript object>

In [ ]:
def l1(u, v):
    """
    Finds the L1 distance between two vectors u and v are 1-dimensional np.array objects
    """
    return np.sum(abs(u - v))

<IPython.core.display.Javascript object>

In [ ]:
def load_data(filename):
    """
    Loads the data into a np array, where each row corresponds to an image patch -- this step is sort of slow.
    Each row in the data is an image, and there are 400 columns.
    """
    return np.genfromtxt(filename, delimiter=',')

<IPython.core.display.Javascript object>

In [ ]:
def create_function(dimensions: List[int], thresholds: List[float]):
    """
    Returns a hash function that computes a hash key from a vector using a set of dimensions and thresholds.
    :param dimensions: A list of dimensions to consider when creating the hash key.
    :param thresholds: A list of threshold values to compare against the corresponding dimensions in the vector.
    :return: A hash function that takes a vector as input and returns a hash key.
    """
    def hash_function(vector):
        bool_array = [vector[dim] >= threshold for dim, threshold in zip(dimensions, thresholds)]
        return "".join(map(str, map(int, bool_array)))
    return hash_function


<IPython.core.display.Javascript object>

In [ ]:
def create_functions(k, L, num_dimensions=400, min_threshold=0, max_threshold=255):
    """
    Creates the LSH functions (functions that compute L K-bit hash keys).
    Each function selects k dimensions (i.e. column indices of the image matrix) at random, and then chooses a random threshold for each dimension, between 0 and
    255.  For any image, if its value on a given dimension is greater than or equal to
    the randomly chosen threshold, we set that bit to 1.  Each hash function returns
    a length-k bit string of the form "0101010001101001...", and the L hash functions 
    will produce L such bit strings for each image.
    """
    functions = []
    for i in range(L):
        dimensions = np.random.randint(low = 0, 
                                   high = num_dimensions,
                                   size = k)
        thresholds = np.random.randint(low = min_threshold, 
                                   high = max_threshold + 1, 
                                   size = k)

        functions.append(create_function(dimensions, thresholds))
    return functions

<IPython.core.display.Javascript object>

In [ ]:
def hash_vector(functions, v):
    """
    Hashes an individual vector (i.e. image). 
    This produces an array with L entries, where each entry is a string of k bits.
    """
    return np.array([f(v) for f in functions])

<IPython.core.display.Javascript object>

In [ ]:
def hash_data(functions, A):
    """
    Hashes the data in A, where each row is a datapoint, using the L functions in "functions."
    """
    return np.array(list(map(lambda v: hash_vector(functions, v), A)))

<IPython.core.display.Javascript object>

In [ ]:
def get_candidates(hashed_A, hashed_point, query_index):
    """
    Retrieve all of the points that hash to one of the same buckets 
    as the query point.  Do not do any random sampling (unlike what the first part of this problem prescribes).
    Don't retrieve a point if it is the same point as the query point.
    """
    candidate_indices = []
    for i, hashed_data_point in enumerate(hashed_data):
        if i != query_index and np.array_equal(hashed_query, hashed_data_point):
            candidate_indices.append(i)
    return candidate_indices

<IPython.core.display.Javascript object>

In [ ]:
def lsh_setup(A, k = 24, L = 10):
    """
    Sets up the LSH.  You should try to call this function as few times as 
    possible, since it is expensive.
    A: The dataset.
    Return the LSH functions and hashed data structure.
    """
    functions = create_functions(k = k, L = L)
    hashed_A = hash_data(functions, A)
    return (functions, hashed_A)

<IPython.core.display.Javascript object>

In [ ]:
def lsh_search(A, hashed_A, functions, query_index, num_neighbors = 10):
    """
    Run the entire LSH algorithm
    """
    hashed_point = hash_vector(functions, A[query_index, :])
    candidate_row_nums = get_candidates(hashed_A, hashed_point, query_index)
    
    distances = map(lambda r: (r, l1(A[r], A[query_index])), candidate_row_nums)
    best_neighbors = sorted(distances, key=lambda t: t[1])[:num_neighbors]

    return [t[0] for t in best_neighbors]

<IPython.core.display.Javascript object>

In [ ]:
def plot(A, row_nums, base_filename):
    """
    Plots images at the specified rows and saves them each to files.
    """
    for row_num in row_nums:
        patch = np.reshape(A[row_num, :], [20, 20])
        im = Image.fromarray(patch)
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.save(base_filename + "-" + str(row_num) + ".png")

<IPython.core.display.Javascript object>

In [ ]:
def linear_search(A, query_index, num_neighbors):
    """
    Finds the nearest neighbors to a given vector, using linear search.
    """
    neighbors_arr = []
    vector = A[query_index]
    for row_index in range(len(A)):
        if row_index != query_index:
            candidate = A[row_index]
            distance = l1(candidate,vector)
            if len(neighbors_arr) < num_neighbors:
                neighbors_arr.append([row_index,distance])
                max_neighbor = max(neighbors_arr,key=lambda x: x[1])
                neighbors_arr = sorted(neighbors_arr, key=lambda x: x[1])
            else:
                if distance > neighbors_arr[-1][1]:
                    continue
                candidate_idx = -1
                for idx in range(len(neighbors_arr),0,-1):
                    if neighbors_arr[idx-1][1] > distance:
                        candidate_idx = idx-1
                neighbors_arr.pop(candidate_idx)
                neighbors_arr.insert(candidate_idx,[row_index,distance])
    sorted_arr = sorted(neighbors_arr, key=lambda x: x[1])
    return [neighbor[0] for neighbor in sorted_arr]

<IPython.core.display.Javascript object>

In [ ]:
def get_error(A, image_patches, linear_neighbors, lsh_neighbours):
    """
    Computes the error measure
    """
    error_sum = 0
    for j in range(10):
        patch = image_patches[j]
        linear_distance = sum([l1(A[idx],A[patch]) for idx in linear_neighbors[j]])
        lsh_distance = sum([l1(A[idx],A[patch]) for idx in lsh_neighbours[j]])
        error_sum += lsh_distance/linear_distance
    return error_sum/10

<IPython.core.display.Javascript object>

In [ ]:
def run_simulation(data,K,L):
    linear_neighbors = []
    lsh_neighbours = []
    functions, hashed_A = lsh_setup(data,K,L)
    for i in [99,199,299,399,499,599,699,799,899,999]:
        linear_neighbors.append(linear_search(data, i, 3))
        lsh_neighbours.append(lsh_search(data,hashed_A,functions,i,3))
    return linear_neighbors,

<IPython.core.display.Javascript object>

In [ ]:
start_time = time.time()
data = load_data("/content/patches.csv")
print("data loaded in %s seconds" % (time.time() - start_time))

In [ ]:
# Test average search time
functions, hashed_A = lsh_setup(data)
linear_time = 0.0
lsh_time = 0.0
for i in [99,199,299,399,499,599,699,799,899,999]:
    print("row: %s"%i)
    start_time = time.time()
    neighbors = linear_search(data, i, 3)
    run_time = time.time() - start_time
    print("neighbors: {0} found in {1} seconds".format(neighbors,run_time))
    linear_time += run_time

    start_time = time.time()
    neighbors = lsh_search(data,hashed_A,functions,i,3)
    run_time = time.time() - start_time
    print("neighbors: {0} found in {1} seconds".format(neighbors,run_time))
    lsh_time += run_time
print('')
print("linear average: {0}".format(linear_time/10))
print("lsh average: {0}".format(lsh_time/10)) 

In [ ]:
# Plot error to L
image_patches = [99,199,299,399,499,599,699,799,899,999]
error_arr = []

for L in range(10,22,2):
    linear_neighbors,lsh_neighbours = run_simulation(data,24,L)
    error_arr.append(get_error(data, image_patches, linear_neighbors, lsh_neighbours))
L = [l for l in range(10,22,2)]
plt.clf() 
plt.close()
plt.plot(L,error_arr)
plt.xlabel('L')
plt.ylabel('error')
plt.savefig('L_error.png')

In [ ]:
# Plot error to k
error_arr = []
for k in range(16,26,2):
    linear_neighbors,lsh_neighbours = run_simulation(data,k,10)
    error_arr.append(get_error(data, image_patches, linear_neighbors, lsh_neighbours))
K = [k for k in range(16,26,2)]
print(K)
print(error_arr)
plt.plot(K,error_arr)
plt.xlabel('K')
plt.ylabel('error')
plt.savefig('K_error.png')